to-do:
1. remove outliers from numercials
2. give order to label encoded
3. use the nice way to concat the two and use the drop_first = True + drop rows if have na's (X.isna().any())
        X = pd.concat([pd.get_dummies(X[categoricalX.columns],drop_first=True),
               remove_outliers(X[numericalX.columns])],axis=1)
        na_idcs = X[X.isna().any(axis=1)].index
        X = pd.DataFrame(X).drop(na_idcs)
        y = pd.DataFrame(y).drop(na_idcs)
        
4. mark all features that are not statistically significant (check the p-value)
5. make graphics of the data even before modelling
6. clean up the notebook and make relevant, contexting notes
7. get the correlation metrics, there are three corr(), heatmap and some pairplot
    6.a. learn how to scatterplot fast -> 
        plt.scatter(x=hk_df['med_fam_income'], y = hk_df['avg_household_income'],c = 'g')
        plt.show()
    6.b. remove highly correlated columns
8. make the distribution plots for each numerical and give comments
    sns.distplot((hk_df['median_home_val']), bins=20)
9. use seaborn
10. make sure to understand which libraries have been imported
11. learn how to use the other transformers (we use a power transformer to remove the skew and rescale the data to have zero mean and unit variance)
12. what is this reshape doing and when to not forget it
13. what hypothesis test one might come up with 
14. see what else we did in week 2 (pandas) and week 4 (modelling)



In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import boxcox
from scipy.stats import norm
import math
%matplotlib inline
pd.options.display.max_rows = 100
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'
import statistics as st
import random
import sklearn
import statsmodels.api as sma

# Import data

In [2]:
df = pd.read_csv("Data/Data_Marketing_Customer_Analysis_Round3.csv")

In [3]:
df = df.drop('effective_to_date', axis = 1)

In [4]:
numericals = df.select_dtypes(np.number)
categoricals = df.select_dtypes(np.object)

/var/folders/wq/2vxkh_6d4g7dpxkr5q82nxzh0000gn/T/ipykernel_49747/3149187810.py:2: DeprecationWarning: `np.object` is a deprecated alias for the builtin `object`. To silence this warning, use `object` by itself. Doing this will not modify any behavior and is safe. 
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  categoricals = df.select_dtypes(np.object)


# Preprocessing

### Check correlation

In [5]:
df.corr()
# no high correlation

,customer_lifetime_value,income,monthly_premium_auto,months_since_last_claim,months_since_policy_inception,number_of_open_complaints,number_of_policies,total_claim_amount
customer_lifetime_value,1.000000,0.003732,0.404235,0.011912,0.017055,-0.032371,0.016789,0.232849
income,0.003732,1.000000,-0.002431,-0.026564,0.003846,0.018903,-0.004461,-0.111221
monthly_premium_auto,0.404235,-0.002431,1.000000,0.010036,0.019696,-0.008811,-0.018805,0.630149
months_since_last_claim,0.011912,-0.026564,0.010036,1.000000,-0.034086,0.001204,0.000814,0.013874
months_since_policy_inception,0.017055,0.003846,0.019696,-0.034086,1.000000,0.002554,-0.009719,0.004186
number_of_open_complaints,-0.032371,0.018903,-0.008811,0.001204,0.002554,1.000000,-0.000303,-0.010668
number_of_policies,0.016789,-0.004461,-0.018805,0.000814,-0.009719,-0.000303,1.000000,-0.008019
total_claim_amount,0.232849,-0.111221,0.630149,0.013874,0.004186,-0.010668,-0.008019,1.000000


### Label Encoding

In [6]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()

categoricals.coverage = le.fit_transform(categoricals.coverage)
categoricals.education = le.fit_transform(categoricals.education)
categoricals.vehicle_class = le.fit_transform(categoricals.vehicle_class)
categoricals.vehicle_size = le.fit_transform(categoricals.vehicle_size)

### Hot Encoding

In [7]:
categoricals = pd.get_dummies(categoricals, prefix = ['month', 'region','response', 'employment_status', 'gender', 'location_code', 'policy_type', 'marital_status','policy', 'renew_offer_type', 'sales_channel'], columns = ['month', 'region','response', 'employment_status', 'gender', 'policy_type', 'location_code', 'marital_status', 'policy', 'renew_offer_type', 'sales_channel'], drop_first=False)

In [ ]:
categoricals

### Concatanation

In [8]:
df = pd.concat([categoricals, numericals], axis=1)

In [9]:
df

,coverage,education,vehicle_class,vehicle_size,month_feb,month_jan,region_central,region_east,region_north west,region_west region,...,sales_channel_call center,sales_channel_web,customer_lifetime_value,income,monthly_premium_auto,months_since_last_claim,months_since_policy_inception,number_of_open_complaints,number_of_policies,total_claim_amount
0,0,1,0,1,1,0,1,0,0,0,...,0,0,4809,48029,61,7,52,0,9,292
1,0,1,0,1,0,1,0,0,0,1,...,1,0,2228,92260,64,3,26,0,1,744
2,0,0,4,1,1,0,0,1,0,0,...,1,0,14947,22139,100,34,31,0,2,480
3,1,1,0,1,0,1,0,0,1,0,...,0,0,22332,49078,97,10,3,0,2,484
4,2,0,0,1,0,1,0,0,1,0,...,0,0,9025,23675,117,33,31,0,7,707
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10684,2,0,1,1,0,1,1,0,0,0,...,0,1,15563,61541,253,12,40,0,7,1214
10685,0,1,0,1,0,1,0,0,1,0,...,0,0,5259,61146,65,7,68,0,6,273
10686,1,0,2,1,1,0,1,0,0,0,...,0,1,23893,39837,201,11,63,0,2,381
10687,2,1,4,1,1,0,0,0,0,1,...,0,0,11971,64195,158,0,27,4,6,618


In [11]:
# remove outliers
from scipy.stats import iqr
def remove_outliers(df):
    for c in df.columns:
            pct_95 = np.percentile(df[c], 95)
            pct_5 = np.percentile(df[c], 5)
            upper_bound = pct_95 + 1.9*iqr(df[c])
            lower_bound = pct_5 - 1.9*iqr(df[c])
            condition = (df[c] < upper_bound) & (df[c] > lower_bound)
            df[c] = df[c][condition]  # Filter out the outliers
    return df

In [12]:
remove_outliers(df)

,coverage,education,vehicle_class,vehicle_size,month_feb,month_jan,region_central,region_east,region_north west,region_west region,...,sales_channel_call center,sales_channel_web,customer_lifetime_value,income,monthly_premium_auto,months_since_last_claim,months_since_policy_inception,number_of_open_complaints,number_of_policies,total_claim_amount
0,0,1,0,NaN,1,0,1,NaN,0,0,...,NaN,NaN,4809.0,48029,61.0,7,52,NaN,NaN,292.0
1,0,1,0,NaN,0,1,0,NaN,0,1,...,NaN,NaN,2228.0,92260,64.0,3,26,NaN,1.0,744.0
2,0,0,4,NaN,1,0,0,NaN,0,0,...,NaN,NaN,14947.0,22139,100.0,34,31,NaN,2.0,480.0
3,1,1,0,NaN,0,1,0,NaN,1,0,...,NaN,NaN,NaN,49078,97.0,10,3,NaN,2.0,484.0
4,2,0,0,NaN,0,1,0,NaN,1,0,...,NaN,NaN,9025.0,23675,117.0,33,31,NaN,7.0,707.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10684,2,0,1,NaN,0,1,1,NaN,0,0,...,NaN,NaN,15563.0,61541,NaN,12,40,NaN,7.0,NaN
10685,0,1,0,NaN,0,1,0,NaN,1,0,...,NaN,NaN,5259.0,61146,65.0,7,68,NaN,6.0,273.0
10686,1,0,2,NaN,1,0,1,NaN,0,0,...,NaN,NaN,NaN,39837,NaN,11,63,NaN,2.0,381.0
10687,2,1,4,NaN,1,0,0,NaN,0,1,...,NaN,NaN,11971.0,64195,158.0,0,27,NaN,6.0,618.0


In [18]:
df.region_east.describe()

count    0.0
mean     NaN
std      NaN
min      NaN
25%      NaN
50%      NaN
75%      NaN
max      NaN
Name: region_east, dtype: float64

In [ ]:
# if I use this function, a lot of NaN appear. Shall they then be removed?

# Model

### X-y split & train and test split 

In [ ]:
# x-y split
X = df.drop('total_claim_amount', axis=1)
y = df.total_claim_amount

In [19]:
# train-test split
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
X_train = pd.DataFrame(X_train)
X_test = pd.DataFrame(X_test)

NameError: name 'X' is not defined

### Standardscaler on numericals

In [ ]:
# scale train and test (fit & transform train; only transform test)
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train) # the standard scaler will only work with numericals
X_test_scaled = scaler.transform(X_test)

In [ ]:
# train model
from sklearn.linear_model import LinearRegression
lr = LinearRegression()
lr.fit(X_train_scaled, y_train)
lr.coef_
lr.intercept_

In [ ]:
# predict
y_lr_train_pred =  pd.DataFrame(lr.predict(X_train_scaled), columns = ['total_claim_amount'])
y_lr_test_pred = pd.DataFrame(lr.predict(X_test_scaled), columns = ['total_claim_amount'])

In [ ]:
# model performance
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error

lr_train_mse = mean_squared_error(y_train, y_lr_train_pred)
lr_train_mae = mean_absolute_error(y_train, y_lr_train_pred)
lr_train_r2 = r2_score(y_train, y_lr_train_pred)
lr_train_ar2 = 1 - (1 - lr_train_r2) * (len(y_train) -1) / (len(y_train) - X_train.shape[1] - 1)
    # has X_train to be the scaled one?

lr_test_mse = mean_squared_error(y_test, y_lr_test_pred)
lr_test_mae = mean_absolute_error(y_test, y_lr_test_pred)
lr_test_r2 = r2_score(y_test, y_lr_test_pred)
lr_test_ar2 = 1 - (1 - lr_test_r2) * (len(y_test) -1) / (len(y_test) - X_test.shape[1] - 1)


lr_results = pd.DataFrame(['Linear Regression', lr_train_mse, lr_test_mse, lr_train_mae, lr_test_mae, lr_train_r2, lr_test_r2, lr_train_ar2, lr_test_ar2]).transpose()
lr_results.columns = ['Method', 'Training MSE', 'Test MSE', 'Training MAE', 'Test MAE', 'Training R2', 'Test R2', 'Training Ad. R2', 'Test Ad. R2']

lr_results
# error_train < error_test => model ok!

In [ ]:
import statsmodels.api as sm

X_train_const_ct = sm.add_constant(X_train_scaled) # adding a constant
model = sm.OLS(y_train, X_train_const_ct).fit()
predictions_train = model.predict(X_train_const_ct)


X_test_const_ct = sm.add_constant(X_test_scaled) # adding a constant
predictions_test = model.predict(X_test_const_ct)
print_model = model.summary()
print(print_model)

## Visualisations

In [ ]:
# scatterplot y_test, y_pred
plt.figure(figsize=(5,5))
plt.scatter(y_test, y_lr_test_pred, alpha = 0.7)

In [ ]:
# histogram residuals
residuals = y_test - y_lr_test_pred.total_claim_amount
residuals.hist()

In [ ]:
# scatterplot residuals
y_test = y_test.to_numpy().reshape(-1,1)

residuals_error = y_lr_test_pred.to_numpy() - y_test
plt.scatter(y_lr_test_pred, residuals_error)

In [ ]:
features_importances = pd.DataFrame(data={
    'Attribute': X_train.columns,
    'Importance': abs(lr.coef_.reshape(len(X_train.columns),))
})
features_importances = features_importances.sort_values(by='Importance', ascending=False)
features_importances

In [ ]:
plt.bar(x=features_importances['Attribute'].iloc[:50], height=features_importances['Importance'].iloc[:50], color='green')
plt.title('Feature importance rankings', size=12)
plt.xticks(rotation='vertical')
plt.show()

In [ ]:
# off-topic

# df['region'] = df['region'].cat.rename_categories({'central': 0, 'west region': 1, 'east': 2, 'north west': 3})
# categoricals = df.select_dtypes(include=['category'])
# for col in df.columns:
#    if df[col].dtype == 'category':
#        df[col] = le.fit_transform(df[col])
# change categories quickly
# for col in ['region', 'response', 'coverage',
#       'education', 'month', 'employment_status',
#       'gender', 'location_code', 'marital_status',
#       'policy_type', 'policy', 'renew_offer_type',
#       'sales_channel', 'vehicle_class', 'vehicle_size']: df[col] = df[col].astype('category')